In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 16 17:14:42 2019

@author: mjofre
"""
###############################################################################
# Script to analyze and compute the Secure Key Rate of the BB84+Decoy-state transmission.
# Whenever possible it will only be true for protocols with decoy-state, polarization encoded.
###############################################################################
import numpy as np
import math

def PrecisionElementsTransmissionBB84DecoyStateComputeSKRdiscretephase(SourcePulseRate,SinglePhotonSourceAttdB,DecoyStatesRates,DecoyStatesPhotPulse,AttenuationdBforDistance,ConfigurationLaserSource,ConfigurationPulseShaping,ConfigurationPhases,ConfigurationPolarizationSelection,ConfigurationSPlevelAtt,SystemDetectorPerformanceArray,DVQKDprotocol):
    # Z. Cao, et al., "Discrete-phase-randomized coherent state source and its application in quantum key distribution", Nwe J. Phys., 17, 053014 (2015). 
    DeltaPhaseDeviation=np.float(ConfigurationPhases[1]*2.0*np.pi)
    Nphases=int(np.min([16,np.max([1.0,ConfigurationPhases[0]])]))
    AmplitudeDeviationPulseShape=np.max([0.0,np.min([1.0,ConfigurationPulseShaping[0]])])
    AmplitudeVariationPulseShape=np.max([0.0,np.min([1.0,ConfigurationPulseShaping[1]])])
    AmplitudeDeviationPulsePickUp=np.max([0.0,np.min([1.0,ConfigurationPolarizationSelection[0]])]) # Quality of the PER
    AmplitudeVariationPulsePickUp=np.max([0.0,np.min([1.0,ConfigurationPolarizationSelection[1]])])
    PowerDeviationPulseSPatt=np.max([0.0,np.min([1.0,ConfigurationSPlevelAtt[0]])])
    PowerVariationPulseSPatt=np.max([0.0,np.min([1.0,ConfigurationSPlevelAtt[1]])])
    PowerDeviationLaser=np.max([0.0,np.min([1.0,ConfigurationLaserSource[0]])])
    PowerVariationLaser=np.max([0.0,np.min([1.0,ConfigurationLaserSource[1]])])

    Y0=SystemDetectorPerformanceArray[0]
    e0=SystemDetectorPerformanceArray[1]
    transmittanceBob=SystemDetectorPerformanceArray[2]
    attDetectors=SystemDetectorPerformanceArray[3]
    TimeJitterStd=SystemDetectorPerformanceArray[4]
    
    Nlen=len(AttenuationdBforDistance)
    RateAlice=SourcePulseRate
    alphaLink=0.1 # 0.1 dB/Km link loss free-space attenuation # 0.2dB/Km for 1550nm telecom fiber
    Distance=AttenuationdBforDistance/alphaLink*1000.0#np.linspace(0,350000,len(Measured_EMu))
    #Distance_plot=Distance*1e-3
    
    MuRate=DecoyStatesRates[0] # Rate of signal states sent
    Nu1Rate=DecoyStatesRates[1] # Rate of decoy 1 states sent
    Nu2Rate=DecoyStatesRates[2] # Rate of decoy 2 states sent
    
    if ((MuRate+Nu1Rate+Nu2Rate)!=1.0):
        print('MuRate+Nu1Rate+Nu2Rate: '+str(MuRate+Nu1Rate+Nu2Rate))
        print('Error in the settings rates of the signal and decoy rates (MuRate+Nu1Rate+Nu2Rate)!=1.0')
        return
    
    fe=1.16 # bidirectional error correction factor
    
    attLink=np.power(10,(-(alphaLink/10.0)*(Distance/1000.0))) # Channel transmittance
    attBob=attDetectors*transmittanceBob # denote the transmittance in Bob's side: internal transmittance of optical components and detector efficiency
    att=attLink*attBob#*(1-1/50); # Overall transmission and detection efficiency between Alice and Bob

    Mu=DecoyStatesPhotPulse[0]*(1.0+AmplitudeDeviationPulseShape)*(1.0+PowerDeviationLaser)*(1.0+PowerDeviationPulseSPatt)#0.5
    Nu1=DecoyStatesPhotPulse[1]*(1.0+AmplitudeDeviationPulseShape)*(1.0+PowerDeviationLaser)*(1.0+PowerDeviationPulseSPatt)#Mu*np.power(10,(-4.65/10.0))
    Nu2=DecoyStatesPhotPulse[2]*(1.0+AmplitudeDeviationPulseShape)*(1.0+PowerDeviationLaser)*(1.0+PowerDeviationPulseSPatt)#Mu*np.power(10,(-14.76/10.0))

    if (TimeJitterStd<=0.0):
      ERFCjitter=0.0
    else:
      ERFCjitter=math.erfc((1.0/SourcePulseRate)/TimeJitterStd)
    
    Y0plusTimeJitter=Y0+ERFCjitter*np.exp(-att*Mu)

    # Yield of a i-photon pulse
    Y1 = Y0plusTimeJitter+(1-(1-att)**1)
    Y2 = Y0plusTimeJitter+(1-(1-att)**2)
    
    # Fung, C.-H.F., Tamaki, K., Lo, H.-K.: Performance of two quantum-key-distribution protocols. Phys. Rev. A 73(1), 012337 (2006)
    # Ma, X., Qi, B., Zhao, Y., Lo, H.-K.: Practical decoy state for quantum key distribution. Phys. Rev. A 72(1), 012326 (2005)
    # Gain of states
    QMu = Y0plusTimeJitter+1-np.exp(-att*Mu) # Gain of the signal state. Equivalent to Y0 if Mu=0
    QNu1 = Y0plusTimeJitter+1-np.exp(-att*Nu1) # Gain of the decoy state. Equivalent to Y0 if Nu1=0
    QNu2 = Y0plusTimeJitter+1-np.exp(-att*Nu2) # Gain of the vacuum state. Equivalent to Y0 if Nu2=0

    if (AmplitudeVariationPulsePickUp<=0.0):
      ERFCcodingState=0.0
    else:
      ERFCcodingState=math.erfc(1.0/AmplitudeVariationPulsePickUp)
    
    edetector=1.0-np.cos(np.arctan(AmplitudeDeviationPulsePickUp+ERFCcodingState))**2.0#0.01; # Probability that a photon hits the wrong detector. It has to due with misalignment and finite PER of polarization splitters
    EMu = (e0*Y0plusTimeJitter+edetector*(1-np.exp(-att*Mu)))/QMu # QBER for the signal state
    ENu1 = (e0*Y0plusTimeJitter+edetector*(1-np.exp(-att*Nu1)))/QNu1 # QBER for the decoy state
    ENu2 = (e0*Y0plusTimeJitter+edetector*(1-np.exp(-att*Nu2)))/QNu2 # QBER for the vacuum state
    
    RateOutputAlice_signal=RateAlice*Mu*(1.0-ERFCjitter)*(1.0-edetector) # Rate of signals states sent out by Alice
    RKR_plot=RateOutputAlice_signal*att # RKR due to signal states

    ## Computations theoretical
    #print('Nu1: '+str(Nu1))
    #print('Nu2: '+str(Nu2))
    #print('QNu1: '+str(QNu1))
    #print('QNu2: '+str(QNu2))
    #############################################################################
    # From Z. Cao, et al., "Discrete-phase-randomized coherent state source and its application in quantum key distribution", Nwe J. Phys., 17, 053014 (2015). 
    # https://iopscience.iop.org/article/10.1088/1367-2630/17/5/053014/pdf
    # Parameters that deviate in the decoy-state protocol from discrete phase are Y and e
    NiIterInfApprox=11 # Simulation number of iterations. The larger the better before saturating the computer
    
    P0=0.0 # A probability of pulse acceptance for vacuum concept derived from Passive Decoy-State Quantum Key Distribution with Coherent Light by Marcos Curty,Marc Jofre,Valerio Pruneri and Morgan W. Mitchell  Entropy 2015, 17(6), 4064-4082; https://doi.org/10.3390/e17064064
    for iIterInfApprox in range(0,NiIterInfApprox,1):
        P0=P0+np.float(((np.power(Mu*(1-(AmplitudeVariationPulseShape)**1)*(1-(PowerVariationPulseSPatt)**1)*(1-(PowerVariationLaser)**1),iIterInfApprox*Nphases+0+(DeltaPhaseDeviation))*np.exp(-Mu*(1+AmplitudeVariationPulseShape**1)*(1+PowerVariationPulseSPatt**1)*(1+PowerVariationLaser**1)))/(np.math.factorial(iIterInfApprox*Nphases+0))))
    
    if (P0<0.0):
        P0=0.0
    
    if (P0>1.0):
        P0=1.0
    
    P1=0.0 # A probability of pulse acceptance for single photon pulses
    for iIterInfApprox in range(0,NiIterInfApprox,1):
        P1=P1+np.float(((np.power(Mu*(1-(AmplitudeVariationPulseShape)**1)*(1-(PowerVariationPulseSPatt)**1)*(1-(PowerVariationLaser)**1),iIterInfApprox*Nphases+1+(DeltaPhaseDeviation))*np.exp(-Mu*(1+AmplitudeVariationPulseShape**1)*(1+PowerVariationPulseSPatt**1)*(1+PowerVariationLaser**1)))/(np.math.factorial(iIterInfApprox*Nphases+1))))
        #print('P1: '+str(P1))
    
    if (P1<0.0):
        P1=0.0
    
    if (P1>1.0):
        P1=1.0
    
    F0=0.0 # Fidelity (between 0 and 1)
    F0numerator=0.0
    F0denominator=0.0
    for iIterInfApprox in range(0,NiIterInfApprox,1):
        F0numerator=F0numerator+np.float(((np.power(Mu,iIterInfApprox*Nphases+0))/(np.math.factorial(iIterInfApprox*Nphases+0)))*(np.power(2.0,-(iIterInfApprox*Nphases+0)/2.0))*(np.cos((iIterInfApprox*Nphases+0)*np.pi/4.0)+np.sin((iIterInfApprox*Nphases+0)*np.pi/4.0)))
        F0denominator=F0denominator+np.float((np.power(Mu,iIterInfApprox*Nphases+0))/np.math.factorial(iIterInfApprox*Nphases+0))
    
    F0=np.float(F0numerator/F0denominator)
    
    # The phase might depend on np.abs(np.exp((1-np.exp(1j*(DeltaPhaseDeviation/N)**2))))
    # The fidelity accounts for an extra factor power 2
    # For the four pick-up modulators, it is accounted for four of them with the fourth power.
    #SourceSinglePhotLevelAttLinear=np.power(10.0,-SinglePhotonSourceAttdB/10.0)
    TotalLossFidelity=1.0#np.sqrt((np.exp((-(((Mu**2.0)*(1.0-np.exp(DeltaPhaseDeviation/Nphases)))**2))))*(np.exp((-((Mu**2.0)*(AmplitudeVariationPulseShape))**4)))*(np.exp((-((Mu**2.0)*(AmplitudeVariationPulsePickUp))**4))**4)*(np.exp((-((Mu**2.0)*(PowerVariationPulseSPatt))**2)))*(np.exp((-((Mu**2.0)*(PowerVariationLaser))**2)))) # I believe variations are encoded into fidelity (constant offsets are encoded somewhere else)
    
    F0=F0*TotalLossFidelity
    
    if (F0<0.0):
        F0=0.0
    
    if (F0>1.0):
        F0=1.0
    
    #print('F0: '+str(F0))
    
    F1=0.0 # Fidelity (between 0 and 1)
    F1numerator=0.0
    F1denominator=0.0
    for iIterInfApprox in range(0,NiIterInfApprox,1):
        F1numerator=F1numerator+np.float(((np.power(Mu,iIterInfApprox*Nphases+1))/(np.math.factorial(iIterInfApprox*Nphases+1)))*(np.power(2.0,-(iIterInfApprox*Nphases+1)/2.0))*(np.cos((iIterInfApprox*Nphases+1)*np.pi/4.0)+np.sin((iIterInfApprox*Nphases+1)*np.pi/4.0)))
        F1denominator=F1denominator+np.float((np.power(Mu,iIterInfApprox*Nphases+1))/np.math.factorial(iIterInfApprox*Nphases+1))
    
    F1=np.float(F1numerator/F1denominator)
    
    F1=F1*TotalLossFidelity
    
    #print('F1: '+str(F1))
    
    if (F1<0.0):
        F1=0.0
    
    if (F1>1.0):
        F1=1.0
    
    F2=0.0 # Fidelity (between 0 and 1)
    F2numerator=0.0
    F2denominator=0.0
    for iIterInfApprox in range(0,NiIterInfApprox,1):
        F2numerator=F2numerator+np.float(((np.power(Mu,iIterInfApprox*Nphases+2))/(np.math.factorial(iIterInfApprox*Nphases+2)))*(np.power(2.0,-(iIterInfApprox*Nphases+2)/2.0))*(np.cos((iIterInfApprox*Nphases+2)*np.pi/4.0)+np.sin((iIterInfApprox*Nphases+2)*np.pi/4.0)))
        F2denominator=F2denominator+np.float((np.power(Mu,iIterInfApprox*Nphases+2))/np.math.factorial(iIterInfApprox*Nphases+2))
    
    F2=np.float(F2numerator/F2denominator)
    
    if (F2<0.0):
        F2=0.0
    
    if (F2>1.0):
        F2=1.0
    
    Y0LowerBound=np.zeros((int(Nlen)),dtype=np.float32)
    Y0LowerBound = (Nu1*QNu2*np.exp(Nu2)-Nu2*QNu1*np.exp(Nu1))/(Nu1-Nu2) # Yield. Changed for N-phase
    if (np.sum(Y0LowerBound<0.0)>0):
        Y0LowerBound[Y0LowerBound<0.0]=0.0
    
    Y1LowerBound=np.zeros((int(Nlen)),dtype=np.float32)
    Y1LowerBound = (np.exp(-Mu))/(Mu*Nu1-Mu*Nu2-Nu1**2+Nu2**2)*(QNu1*np.exp(Nu1)-QNu2*np.exp(Nu2)-(Nu1**2-Nu2**2)/(Mu**2)*(QMu*np.exp(Mu)-Y0LowerBound)) # Yield. Changed for N-phase
    if (np.sum(Y1LowerBound<0.0)>0):
        Y1LowerBound[Y1LowerBound<0.0]=0.0
    
    Y2LowerBound=np.zeros((int(Nlen)),dtype=np.float32)
    Y2LowerBound = Y2# approx #(np.exp(-Mu))/(Mu*Nu1-Mu*Nu2-Nu1**2+Nu2**2)*(QNu1*np.exp(Nu1)-QNu2*np.exp(Nu2)-(Nu1**2-Nu2**2)/(Mu**2)*(QMu*np.exp(Mu)-Y0LowerBound)) # Yield of two-photon pulses
    if (np.sum(Y2LowerBound<0.0)>0):
        Y2LowerBound[Y2LowerBound<0.0]=0.0
    
    Q1LowerBound = Mu*np.exp(-Mu)*Y1LowerBound # Gain of the single photon pulses.
    Q2LowerBound = ((Mu**2)/2)*np.exp(-Mu)*Y2LowerBound # Gain of the two-photon pulses.

    Delta0=np.zeros((int(Nlen)),dtype=np.float32)
    Delta0=(1.0-F0)/(2.0*Y0LowerBound)
    
    Delta0aux=np.zeros((int(Nlen)),dtype=np.float32)
    Delta0aux=Delta0*(1.0-Delta0)*e0*(1.0-e0)
    if (np.sum(Delta0aux<0.0)>0):
        Delta0aux[Delta0aux<0.0]=0.0
    
    e0Bound=e0*np.ones_like(Delta0)#+4.0*Delta0*(1.0-Delta0)*(1.0-2.0*e0)+4.0*(1.0-2.0*Delta0)*np.sqrt(Delta0aux) # Changed for N-phase
    
    if (np.sum(e0Bound<0.0)>0):
        e0Bound[e0Bound<0.0]=0.0
    
    if (np.sum(e0Bound>1.0)>0):
        e0Bound[e0Bound>1.0]=1.0
        
    Delta1=np.zeros((int(Nlen)),dtype=np.float32)
    Delta1=(1.0-F1)/(2.0*Y1LowerBound)   
    
    e1=(EMu*QMu*np.exp(Mu)-e0Bound*Y0LowerBound)/(Q1LowerBound*np.exp(Mu))
    e1=np.abs(e1)
    if (np.sum(e1<0.0)>0):
        e1[e1<0.0]=0.0
    
    if (np.sum(e1>1.0)>0):
        e1[e1>1.0]=1.0
    
    Delta1aux=np.zeros((int(Nlen)),dtype=np.float32)
    Delta1aux=Delta1*(1.0-Delta1)*e1*(1.0-e1)
    if (np.sum(Delta1aux<0.0)>0):
        Delta1aux[Delta1aux<0.0]=0.0
    
    e1UpperBound = e1+4.0*Delta1*(1.0-Delta1)*(1.0-2.0*e1)+4.0*(1.0-2.0*Delta1)*np.sqrt(Delta1aux) # Error rate of single photon pulses. Changed for N-phase
    
    Delta1=np.zeros((int(Nlen)),dtype=np.float32)
    Delta1=(1.0-F2)/(2.0*Y2LowerBound) 

    e2=(EMu*QMu*np.exp(Mu)-e0Bound*Y0LowerBound)/(Q2LowerBound*np.exp(Mu))#1-(1-(EMu*QMu*np.exp(Mu)-e0Bound*Y0LowerBound)/(Q2LowerBound*np.exp(Mu)))**2 # Error rate of two-photon pulses
    e2=np.abs(e2)
    if (np.sum(e2<0.0)>0):
        e2[e2<0.0]=0.0
    
    if (np.sum(e2>1.0)>0):
        e2[e2>1.0]=1.0
    
    Delta1aux=np.zeros((int(Nlen)),dtype=np.float32)
    Delta1aux=Delta1*(1.0-Delta1)*e2*(1.0-e2)
    if (np.sum(Delta1aux<0.0)>0):
        Delta1aux[Delta1aux<0.0]=0.0
    
    e2UpperBound = e2+4.0*Delta1*(1.0-Delta1)*(1.0-2.0*e2)+4.0*(1.0-2.0*Delta1)*np.sqrt(Delta1aux) # Error rate of single photon pulses. Changed for N-phase
    
    if (np.sum(e1UpperBound<0.0)>0):
        e1UpperBound[e1UpperBound<0.0]=0.0
    
    if (np.sum(e1UpperBound>1.0)>0):
        e1UpperBound[e1UpperBound>1.0]=1.0    
    
    if (np.sum(e2UpperBound<0.0)>0):
        e2UpperBound[e2UpperBound<0.0]=0.0
    
    if (np.sum(e2UpperBound>1.0)>0):
        e2UpperBound[e2UpperBound>1.0]=1.0
    
    QBER=e1UpperBound # QBER due only to single-photon
    
    H2EMu=np.zeros(int(len(EMu)),dtype=np.float32)
    for iIter in range(0,len(EMu),1):
        if (EMu[iIter]<=0.0):
            H2EMu[iIter]=0.0
        else:
            H2EMu[iIter] = -EMu[iIter]*np.log2(EMu[iIter])-(1.0-EMu[iIter])*np.log2(1.0-EMu[iIter])
    
    H2e0Upper=np.zeros(int(len(e0Bound)),dtype=np.float32)
    for iIter in range(0,len(e0Bound),1):
        if (e0Bound[iIter]<=0.0):
            H2e0Upper[iIter]=0.0
        else:
            H2e0Upper[iIter] = -e0Bound[iIter]*np.log2(e0Bound[iIter])-(1.0-e0Bound[iIter])*np.log2(1.0-e0Bound[iIter])
    
    H2e1Upper=np.zeros(int(len(e1UpperBound)),dtype=np.float32)
    
    for iIter in range(0,len(e1UpperBound),1):
        if (e1UpperBound[iIter]<=0.0):
            H2e1Upper[iIter]=0.0
        else:
            H2e1Upper[iIter] = -e1UpperBound[iIter]*np.log2(e1UpperBound[iIter])-(1.0-e1UpperBound[iIter])*np.log2(1.0-e1UpperBound[iIter])
    
    H2e2Upper=np.zeros(int(len(e2UpperBound)),dtype=np.float32)
    for iIter in range(0,len(e2UpperBound),1):
        if (e2UpperBound[iIter]<=0.0):
            H2e2Upper[iIter]=0.0
        else:
            H2e2Upper[iIter] = -e2UpperBound[iIter]*np.log2(e2UpperBound[iIter])-(1.0-e2UpperBound[iIter])*np.log2(1.0-e2UpperBound[iIter])
    
    # Lower Secure Key Rate
    if (DVQKDprotocol=='SARG04+decoy'): # One-photon and 2-photon pulses contribute
      CoincidenceSameBasePrepMeas=0.25 # For SARG04 it is 0.25.
      KeyBitRateLowerBond = CoincidenceSameBasePrepMeas*(-QMu*fe*H2EMu + Q1LowerBound*(1.0-H2e1Upper)+Q2LowerBound*(1.0-H2e2Upper))*RateOutputAlice_signal
    else: # for instance 'BB84+decoy' only one-photon pulses contribute
      CoincidenceSameBasePrepMeas=0.5 # For BB84 it is 0.5; for SARG04 it is 0.25.
      KeyBitRateLowerBond = CoincidenceSameBasePrepMeas*(-QMu*fe*H2EMu + Q1LowerBound*(1.0-H2e1Upper))*RateOutputAlice_signal#CoincidenceSameBasePrepMeas*(-QMu*fe*H2EMu + Q0LowerBound*(1.0-H2e0Upper) + Q1LowerBound*(1.0-H2e1Upper))*RateOutputAlice_signal
    
    ## Remove glitches
    if (np.sum(np.logical_or(KeyBitRateLowerBond<0.0,np.logical_or(e1UpperBound>=0.5,H2e1Upper<=0.0)))>0):
        KeyBitRateLowerBond[np.logical_or(KeyBitRateLowerBond<0.0,np.logical_or(e1UpperBound>=0.5,H2e1Upper<=0.0))]=0.0
    
    for iIter in range(1,int(Nlen-2),1):
        if (np.logical_and(KeyBitRateLowerBond[iIter-1]<=0.0,KeyBitRateLowerBond[iIter+1]<=0.0) or np.logical_and(KeyBitRateLowerBond[iIter-2]<=0.0,KeyBitRateLowerBond[iIter+2]<=0.0)):
            KeyBitRateLowerBond[iIter]=0.0    
    
    if (np.sum(KeyBitRateLowerBond<=0.0)>0):
        QBER[KeyBitRateLowerBond<=0.0]=0.5
    
    if (np.sum(QBER>=0.5)>0): # Not to be displayed in the image
        QBER[QBER>=0.5]=0.0
    
    return attLink,RKR_plot,KeyBitRateLowerBond,QBER
    
